In [1]:
%run stdPackages.ipynb
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other'], 
        'variable2D': ['Generators_FuelMix','HourlyVariation'],
        'scalars': ['Scalars'],
        'maps': ['Generators_Categories']}
db = dbFromWB(os.path.join(d['data'],'0_GlobalData.xlsx'), read)
readSets(db)

# The ```mBasicInt_EmissionCap``` model

In [2]:
m = mBasicInt.mBasicInt_EmissionCap(db)
m.solve()

Solution status 0: Optimization terminated successfully.


### **The model**

The model is an extension to the model [mBasicInt](M_mBasicInt.ipynb). The extension is that a cap on $CO_2$ emissions from fuel consumption. Specifically, as also outlined for ```mBasicInt```, fuel consumption and emissions can be computed as:
$$\begin{align}
    \text{FuelConsumption}_{BFt} =& \sum_{id,h}\mu_{BFt}^{id}\cdot E_{id,h} \\ 
    \text{Emissions}_m =& \sum_{BFt}\text{FuelConsumption}_{BFt}\cdot \text{EmissionIntensity}_{BFt,m},
\end{align}$$
where $\mu_{BFt}^{id}$ is the fuel intensity for plant $id$. The emission cap is thus included as the inequality constraint that:
$$\begin{align}
    \sum_{BFt, id, h} \mu_{BFt}^{id}\cdot E_{id, h} \cdot \text{EmissionIntensity}_{BFt,CO2} \leq \text{CO2Cap}
\end{align}$$

### Adjustments to the code

Compared to ```mBasicInt``` the main adjustment is that there is an added ```block``` of the type ```ub```:

<img src="snippets/mBasicInt_EmissionCap_snippet.png" width="1300" height="400">

The specification of the constraint is in terms of syntax equivalent to the ```eq``` constraints:
* There is a constraint named ```emissionsCap``` ,
* with the parameter constraint $b_{ub}$ from ```self.db['CO2Cap']```.
* $A_{ub}$ coefficients for this constraint are on the variable ```'Generation'```. Specifically, the parameters are the emission intensity for CO2 repeated for all hours:

*Parameters for the emission cap:*

In [3]:
lpCompiler.broadcast(mBasicInt.plantEmissionIntensity(m.db).xs('CO2', level = 'EmissionType'), m.db['h'])

id   h
id1  1    0.109699
     2    0.109699
     3    0.109699
     4    0.109699
id2  1    0.072329
     2    0.072329
     3    0.072329
     4    0.072329
id3  1         0.0
     2         0.0
     3         0.0
     4         0.0
id4  1    0.009437
     2    0.009437
     3    0.009437
     4    0.009437
id5  1    0.048875
     2    0.048875
     3    0.048875
     4    0.048875
id6  1         0.0
     2         0.0
     3         0.0
     4         0.0
id7  1         0.0
     2         0.0
     3         0.0
     4         0.0
id8  1         0.0
     2         0.0
     3         0.0
     4         0.0
dtype: object